In [ ]:
import os
import cv2
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from train import Network
from cityscapes import CityscapesDataModule


def process_mask(seg_mask, ds):
    # colored_mask = np.zeros((*image.shape[:2], 3), dtype=np.uint8)
    colored_mask = np.zeros((384, 768, 3), dtype=np.uint8)
    for id, color in enumerate(ds.COLOR_PALETTE):
        colored_mask[seg_mask == id] = ds.COLOR_PALETTE[color]

    return colored_mask


def process_bboxes(image, bboxes, labels, ds):
    for idx, bbox in enumerate(bboxes):
        bbox = bbox.numpy().astype(np.int32)
        label = (
            ds.LOCALIZATION_CLASSES[labels[idx].item()] + " " + str(labels[idx].item())
        )
        color = ds.COLOR_PALETTE[11 + labels[idx].item()]

        font_scale = 0.75
        thickness = 2

        image = cv2.rectangle(
            image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, thickness
        )

        (w, h), _ = cv2.getTextSize(
            label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness
        )
        image = cv2.rectangle(
            image, (bbox[0], bbox[1] - h - 10), (bbox[0] + w, bbox[1]), color, -1
        )
        image = cv2.putText(
            image,
            label,
            (bbox[0], bbox[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            [255, 255, 255],
            thickness,
        )

    return image


def get_detections(centerness, regression, score_threshold=0.25, topk=100):
    b, _, h, w = centerness.shape

    detections = []

    center_max = F.max_pool2d(centerness, kernel_size=3, stride=1, padding=1)
    center_mask = (center_max == centerness).float()
    centerness = centerness * center_mask

    for batch in range(b):

        scores, indices = torch.topk(centerness[batch].view(-1), k=topk)

        scores = scores[scores >= score_threshold]
        topk_indices = indices[: len(scores)]

        # print(len(scores))

        labels = (topk_indices / (h * w)).int()
        indices = topk_indices % (h * w)

        xs_orig = (indices % w).int()
        ys_orig = (indices / w).int()

        wh = regression[batch][:, ys_orig, xs_orig]

        half_w, half_h = wh[0] / 2, wh[1] / 2

        bboxes = (
            torch.stack(
                [
                    xs_orig - half_w,
                    ys_orig - half_h,
                    xs_orig + half_w,
                    ys_orig + half_h,
                ],
                dim=1,
            )
            * 4
        )

        detections.append([bboxes, scores, labels])

        return detections


dm = CityscapesDataModule()
dm.setup()

base_dir = "logs/cityscapes/version_0/checkpoints"

In [ ]:
snapshot = torch.load(os.path.join(base_dir, os.listdir(base_dir)[0]))

print(snapshot["epoch"])

net = Network()
net.load_state_dict(snapshot["state_dict"])
net.eval()

with torch.no_grad():
    for idx, batch in enumerate(dm.val_dataloader()):
        if idx == 100:
            classifier, _, centerness, regression = net.model(batch[0])
            break

image = batch[0][0].numpy()
ds = dm.val_dataset
seg = classifier.argmax(dim=1)[0]
seg = process_mask(seg.numpy(), ds)

detections = get_detections(centerness, regression, score_threshold=0.3, topk=100)

inv_image = (
    (
        (
            (image * np.array((0.229, 0.224, 0.225)).reshape(3, 1, 1))
            + np.array((0.485, 0.456, 0.406)).reshape(3, 1, 1)
        )
        * 255
    )
    .transpose(1, 2, 0)
    .astype(np.uint8)
)
final = process_bboxes(inv_image.copy(), detections[0][0], detections[0][2], ds)

plt.imshow(final)
# plt.imshow(inv_image)
plt.imshow(seg, alpha=0.55)

In [ ]:
plt.imshow(centerness[0][0], cmap="gray")